In [159]:
import csv
all_vars = {}
all_vars['_year'] = 0.0
all_vars['_global_socialism'] = 1.0
all_vars['_global_liberalism'] = 1.0

In [160]:
slider_names  = {} # This pairs the slider objects with their names

class Slider():
    minimum = 0.0
    maximum = 1.0
    diff = None
    def __init__(self, name, identifier, a, b):
        self.name = name
        self.identifier = identifier
        if identifier == "PERCENTAGE":
            self.minimum = float(a)/100.0
            self.maximum = float(b)/100.0
        elif float(a) > 0:
            self.diff = float(a)
        else:
            self.identifier = "PERCENTAGE"
    def slide(self, value):
        if self.identifier == "PERCENTAGE":
            return min(self.maximum, max(self.minimum, value))
        else:
            return min(self.maximum, max(self.minimum, round(value*self.diff)/self.diff))

with open("sliders.csv") as rs:
    csv_reader = csv.reader(rs, delimiter=',')
    for row in csv_reader:
        slider_names[row[1]] = Slider(row[1], row[2], row[3], row[4])

In [161]:
config = {} # Values from simconfig

with open("simconfig.txt") as rs:
    j = 0
    for ln in rs:
        if j == 0:
            j+=1
            continue
        if ln != "[config]" and ln != "\n":
            var_name = ""
            i = 0
            while i < len(ln) and ln[i] != ' ':
                var_name += ln[i]
                i += 1
            i += 3
            value = ""
            while i < len(ln) and ln[i] != '\n' and ln[i] != ';':
                value += ln[i]
                i += 1
            config[var_name] = float(value)
            

In [162]:
policy_names  = {} # This pairs the policy objects with their names
order_of_operations = {'^': 0,
                       '*': 1,
                       '/': 1,
                       '+': 2,
                       '-': 2,
                       None: 3}
active_policies = []

def isOperator(char):
    return True if char in "+-*/^" else False

def isVal(char):
    return False if char in "+-*/^,();" else True

def isFloat(val):
    return (val[0].isdigit() or val[0] == '-') if len(val) else False

def finder(s, var, inertia=1):
    if var == "x":
        return s.getValue()
    else:
        if var in all_vars:
            a = all_vars[var]
            if type(a) == int or type(a) == float:
                return a
            else:
                return a.getValue(inertia)
        else:
            j = 0
            val = ""
            under = ""
            while j < len(var):
                if var[j] == '_':
                    val = under
                    under = ""
                else:
                    under += var[j]
                j += 1
            if under == "cost":
                if val in all_vars:
                    try:
                        return all_vars[val].Cost()
                    except:
                        return 0
                else:
                    raise NameError(val)
            elif under == "income":
                if val in all_vars:
                    try:
                        return all_vars[val].Income()
                    except:
                        # FIX THIS LATER ON
                        # ############# print("a", all_vars[val], val)
                        return 0
                        raise NameError(val)
                else:
                    raise NameError(val)
            elif under == "freq":
                return all_vars[val+"_perc"]
            else:
                # ############# print("b", var)
                return 0

def setEffect(var, value, nxt=False):
    if var not in all_vars:
        j = 0
        val = ""
        under = ""
        while j < len(var):
            if var[j] == '_':
                val = under
                under = ""
            else:
                under += var[j]
            j += 1
        if under == "cost":
            if val in all_vars:
                try:
                    all_vars[val].addCost(value, nxt)
                    # FIX THIS LATER ON
                except:
                    return 0
                    # ############# print('f' + 'RIP')
            else:
                raise NameError(val)
        elif under == "income":
            if val in all_vars:
                try:
                    all_vars[val].addIncome(value, nxt)
                    # FIX THIS LATER ON
                except:
                    return 0
#                     print('e' + 'RIP')
            else:
                raise NameError(val)
        elif under == "freq":
            all_vars[val+"_perc"] += value
        else:
            # FIX THIS LATER ON
            # ############# print("d" + var)
            return 0
    else:
        if type(all_vars[var]) == int or type(all_vars[var]) == float:
            all_vars[var] += value
        else:
            # print("c", all_vars[var].name, value)
            all_vars[var].addEffect(value, nxt)
    
def lambduh(operator):
    if operator == '+':
        return lambda a,b: a + b
    elif operator == '-':
        return lambda a,b: a - b
    elif operator == '*':
        return lambda a,b: a * b
    elif operator == '/':
        return lambda a,b: a / b
    elif operator == '^':
        return lambda a,b: a ** b
    else:
        return None
class All():
    def setValue(self, value, nxt=False):
        if nxt:
            yr = all_vars['_year'] + 1/4
        else:
            yr = all_vars['_year']
        self.value[yr] = float(value)
    def setEffect(self, value, nxt=False):
        if nxt:
            yr = all_vars['_year'] + 1/4
        else:
            yr = all_vars['_year']
        self.effect[yr] = float(value) + 1.0
    def getValue(self, count=1):
        avg = 0
        i = all_vars['_year']
        while i > all_vars['_year'] - count/4:
            if i in self.value:
                avg += self.value[i]*self.effect[i]
            i -= 1/4
        return avg/count
    def addValue(self, value, nxt=False):
        if nxt:
            yr = all_vars['_year'] + 1/4
        else:
            yr = all_vars['_year']
        if yr in self.value:
            self.value[yr] += float(value)
        else:
            self.setValue(float(value), nxt)
    def addEffect(self, value, nxt=False):
        if nxt:
            yr = all_vars['_year'] + 1/4
        else:
            yr = all_vars['_year']
        if yr in self.effect:
            self.effect[yr] += float(value)
        else:
            self.setEffect(float(value), nxt)
            
    
class Policy(All):
    def __init__(self, name, slider, intro, canc, rai, low, min_cost, max_cost, implementation, min_income, max_income):
        self.name = name
        self.multiply = False
        self.slider_name = slider # Slider name
        self.slider = slider_names[slider] # Slider object
        self.introduce = int(intro)
        self.cancel = int(canc)
        self.raiz = int(rai)
        self.lower = int(low)
        self.min_cost = float(min_cost)
        self.max_cost = float(max_cost)
        self.implementation = int(implementation)
        self.min_income = float(min_income)
        self.max_income = float(max_income)
        self.value = {0.0: 0.5}
        self.effect = {0.0: 1.0}
        self.active = False
        self.default_cost = 1.0
        self.default_income = 1.0
        self.cost_multiplier = []
        self.income_multiplier = []
        self.effects = []
        self.cost_multiply = {}
        self.income_multiply = {}
    def update(self):
        self.value[all_vars['_year']] = self.value[all_vars['_year']-1/4]
        self.effect[all_vars['_year']] = 1.0
#         self.income_multiply[all_vars['_year']] = 1.0
#         self.cost_multiply[all_vars['_year']] = 1.0
    def changeValue(self, value):
        self.setValue(self.slider.slide(value))
        self.costMul()
        self.incomeMul()
    def Cost(count=1):
        avg = 0
        i = all_vars['_year']
        while i > all_vars['_year'] - count/4:
            if i in self.value:
                avg += self.value[i]*self.cost_multiply[i]
            i -= 1/4
        return avg/count
    def Income(count=1):
        avg = 0
        i = all_vars['_year']
        while i > all_vars['_year'] - count/4:
            if i in self.value:
                avg += self.value[i]*self.income_multiply[i]
            i -= 1/4
        return avg/count
    def activate(self, boolean):
        if boolean == 1 and not self.active:
            self.active = True
            active_policies.append(self.name)
        elif boolean == 0 and self.active:
            self.active = False
            active_policies.remove(self.name)
    def __str__(self):
        return self.name + " " + self.slider_name
    def realCost(self):
        return self.getCost()*all_vars['wealth_mod']
    def realIncome(self):
        return self.getIncome()*all_vars['wealth_mod']
    def costMul(self):
        # Apply Cost Mul and find the cost to return
        counter = self.default_cost
        for i in self.cost_multiplier:
            if i.target in all_vars:
                if not (type(all_vars[i.target]) == Situations and not all_vars[i.target].started):
                    counter += i.apply()
            else:
                print(i.target)
        self.addCost(counter)
    def incomeMul(self):
        # Apply Income Mul and find the income to return
        counter = self.default_income
        for i in self.income_multiplier:
            if i.target in all_vars and i.target:
                if not (type(all_vars[i.target]) == Situations and not all_vars[i.target].started):
                    if self.multiply:
                        counter *= i.apply()
                    else:
                        counter += i.apply()
            else:
                print(i.target)
        if self.multiply:
            self.mulIncome(counter)
        else:
            self.addIncome(counter)
    def addCost(self, value, nxt=False):
        if nxt:
            yr = all_vars['_year'] + 1/4
        else:
            yr = all_vars['_year']
        if yr in self.cost_multiply:
            self.cost_multiply[yr] += float(value)
        else:
            self.cost_multiply[yr] = float(value)
    def getCost(self):
        return max(self.min_cost, min(self.max_cost, self.value[all_vars['_year']]*self.cost_multiply[all_vars['_year']]*(self.max_cost-self.min_cost)+self.min_cost))
    def addIncome(self, value, nxt=False):
        if nxt:
            yr = all_vars['_year'] + 1/4
        else:
            yr = all_vars['_year']
        if yr in self.income_multiply:
            self.income_multiply[yr] += float(value)
        else:
            self.income_multiply[yr] = float(value)
    def mulIncome(self, value):
        if all_vars['_year'] in self.income_multiply:
            self.income_multiply[all_vars['_year']] *= float(value)
        else:
            self.income_multiply[all_vars['_year']] = float(value)
    def getIncome(self):
        return max(self.min_income, min(self.max_income, self.value[all_vars['_year']]*self.income_multiply[all_vars['_year']]*(self.max_income-self.min_income)+self.min_income))
    def applyEffects(self):
        for i in self.effects:
            setEffect(i.target, i.apply(), True)
            # APPLY TO THE DICTIONARY VERSION

class Effect():
    # [target],[value1][operator1]([value2][operator2][values3])[operator3][value4],[inertia]
    def __init__(self, policy, target, value1, operator1, value2, operator2, value3, operator3 = None, value4 = None, inertia = 1):
        self.target = target
        self.value1 = value1
        self.operator1 = operator1
        self.value2 = value2
        self.operator2 = operator2
        self.value3 = value3
        self.inertia = inertia
        self.operator3 = operator3
        self.value4 = value4
        self.policy = policy
        self.f = self.funcGenerator()
    def getValue(self, inertia = 1):
        return finder(self, self.policy, inertia)
    def funcGenerator(self):
        op1 = lambduh(self.operator1)
        op2 = lambduh(self.operator2)
        op3 = lambduh(self.operator3)
        def function():
            if isFloat(self.value2):
                a2 = float(self.value2)
            else:
                a2 = finder(self, self.value2) #HERE YOU NEED TO REFERENCE THE DICTIONARY THAT THIS WOULD BE STORED IN
                #USE INERTIA
            if isFloat(self.value3):
                b3 = float(self.value3)
            else:
                b3 = finder(self, self.value3) #HERE YOU NEED TO REFERENCE THE DICTIONARY THAT THIS WOULD BE STORED IN
            p_val = op2(a2, b3)
            try:
                if order_of_operations[self.operator1] <= order_of_operations[self.operator3]:
                    if isFloat(self.value1):
                        a1 = float(self.value1)
                    else:
                        a1 = finder(self, self.value1) #HERE YOU NEED TO REFERENCE THE DICTIONARY THAT THIS WOULD BE STORED IN
                    f_val = op1(a1, p_val)
                    if self.operator3:
                        if isFloat(self.value4):
                            b4 = float(self.value4)
                        else:
                            b4 = finder(self, self.value4) #HERE YOU NEED TO REFERENCE THE DICTIONARY THAT THIS WOULD BE STORED IN
                        return op3(f_val, b4)
                    else:
                        return f_val
                else:
                    if isFloat(self.value4):
                        b4 = float(self.value4)
                    else:
                        b4 = finder(self, self.value4) #HERE YOU NEED TO REFERENCE THE DICTIONARY THAT THIS WOULD BE STORED IN
                    f_val = op3(p_val, b4)
                    if isFloat(self.value1):
                        a1 = float(self.value1)
                    else:
                        a1 = finder(self, self.value1) #HERE YOU NEED TO REFERENCE THE DICTIONARY THAT THIS WOULD BE STORED IN
                    return op1(a1, f_val)
            except:
                raise NameError(self)
        return function
    def apply(self):
        # YOU NEED TO CALL FROM DICTIONARY
#         if self.target == 'GDP':
#             print(self.policy)
        return self.f()
    def __str__(self):
        return self.target + " " + self.value1 + self.operator1 + self.value2 + self.operator2 + self.value3

def separateBySC(string):
    arr = []
    temp_string = ""
    i = 0
    while i < len(string):
        if string[i] == ';':
            arr.append(temp_string)
            temp_string = ""
        else:
            temp_string += string[i]
        i += 1
    arr.append(temp_string)
    return arr

def separateByC(string):
    arr = []
    temp_string = ""
    i = 0
    while i < len(string):
        if string[i] == ',':
            arr.append(temp_string)
            temp_string = ""
        else:
            temp_string += string[i]
        i += 1
    arr.append(temp_string)
    return arr

def separateEffect(string):
    arr = []
    i = 0
    init = 0
    a = ""
    while i < len(string) and (isVal(string[i]) or i == init):
        a += string[i]
        i += 1
    arr.append(a)
    arr.append(string[i])
    i += 2
    a = ""
    init = i
    while i < len(string) and (isVal(string[i]) or i == init):
        a += string[i]
        i += 1
    arr.append(a)
    arr.append(string[i])
    i += 1
    a = ""
    while i < len(string) and (isVal(string[i]) or i == init):
        a += string[i]
        i += 1
    arr.append(a)
    if i+1 == len(string):
        return arr
    i += 1
    arr.append(string[i])
    while i < len(string) and (isVal(string[i]) or i == init):
        a += string[i]
        i += 1
    arr.append(a)
    return arr

with open("policies.csv") as rs:
    csv_reader = csv.reader(rs, delimiter=',')
    i = 0
    for row in csv_reader:
        if i > 0:
            policy_names[row[1]] = Policy(row[1], row[3], row[6], row[7], row[8], row[9], row[11], row[12], row[14], row[15], row[16])
            if row[5] == "MULTIPLYINCOME":
                policy_names[row[1]].multiply = True
            if row[5] == "UNCANCELLABLE":
                policy_names[row[1]].activate(1)
            if row[13] != "" and row[13] != " ":
                effect_list = policy_names[row[1]].cost_multiplier
                cm = separateBySC(row[13])
                for s in cm:
                    a = separateByC(s)
                    if a[0] == "_default_":
                        policy_names[row[1]].default_cost = float(a[1])
                    else:
                        e_vars = separateEffect(a[1])
                        if len(e_vars) == 5:
                            effect_list.append(Effect(row[1], a[0], e_vars[0], e_vars[1], e_vars[2], e_vars[3], e_vars[4]))
                        elif len(e_vars) == 7:
                            effect_list.append(Effect(row[1], a[0], e_vars[0], e_vars[1], e_vars[2], e_vars[3], e_vars[4], e_vars[5], e_vars[6]))
            if row[17] != "" and row[17] != " ":
                effect_list = policy_names[row[1]].income_multiplier
                cm = separateBySC(row[17])
                for s in cm:
                    a = separateByC(s)
                    if a[0] == "_default_":
                        policy_names[row[1]].default_income = float(a[1])
                    else:
                        e_vars = separateEffect(a[1])
                        if len(e_vars) == 5:
                            effect_list.append(Effect(row[1], a[0], e_vars[0], e_vars[1], e_vars[2], e_vars[3], e_vars[4]))
                        elif len(e_vars) == 7:
                            effect_list.append(Effect(row[1], a[0], e_vars[0], e_vars[1], e_vars[2], e_vars[3], e_vars[4], e_vars[5], e_vars[6]))
            effect_list = policy_names[row[1]].effects
            for eff in row[19:]:
                if eff == " " or eff == "":
                    continue
                a = separateByC(eff)
                e_vars = separateEffect(a[1])
                if len(e_vars) == 5:
                    effect_list.append(Effect(row[1], a[0], e_vars[0], e_vars[1], e_vars[2], e_vars[3], e_vars[4]))
                elif len(e_vars) == 7:
                    effect_list.append(Effect(row[1], a[0], e_vars[0], e_vars[1], e_vars[2], e_vars[3], e_vars[4], e_vars[5], e_vars[6]))
        else:
            i+=1

In [163]:
simulation_vars = {}

class SimVars(All):
    def __init__(self, name, default, minim, maxim):
        self.name = name
        self.default = float(default)
        self.minim = float(minim)
        self.maxim = float(maxim)
        self.realMinim = float(minim)
        self.realMaxim = float(maxim)
        self.aff = []
        self.eff = []
        self.value = {0.0: float(default)}
        self.effect = {0.0: 1.0}
    def setValue(self, value, nxt=False):
        if nxt:
            yr = all_vars['_year'] + 1/4
        else:
            yr = all_vars['_year']
        #self.value[yr] = max(self.minim, min(self.maxim, value))
    def getValue(self, count=1):
        avg = 0
        i = all_vars['_year']
        while i > all_vars['_year'] - count/4:
            if i in self.value:
                avg += max(self.minim, min(self.maxim, self.value[i]*self.effect[i]))
            i -= 1/4
        return avg/count
    def addValue(self, value, nxt=False):
        if nxt:
            yr = all_vars['_year'] + 1/4
        else:
            yr = all_vars['_year']
        if yr in self.value:
            self.value[yr] += float(value)
            #self.value[yr] = max(self.minim, min(self.maxim, self.value[yr]))
        else:
            self.setValue(self.default + float(value), nxt)
    def applyEffects(self):
        for i in self.eff:
            # apply effects
            setEffect(i.target, i.apply(), True)
    def getRealValue(self):
        return self.value[all_vars['_year']]*self.effect[all_vars['_year']]*(self.realMaxim - self.realMinim) + self.realMinim
    def update(self):
        # Code for Updating Aff and Eff
        self.value[all_vars['_year']] = self.default
        self.effect[all_vars['_year']] = 1.0
        for i in self.aff:
            try:
                #print("before", self.getValue())
                self.addValue(i.apply())
                #print("after", self.getValue())
            except:
                raise NameError(i.target)
        self.applyEffects()
            # add effect

with open("simulation.csv") as rs:
    csv_reader = csv.reader(rs, delimiter=',')
    i = 0
    for row in csv_reader:
        if i > 0:
            simulation_vars[row[1]] = SimVars(row[1], row[5], row[6], row[7])
            e = simulation_vars[row[1]].aff
            pol = True
            for s in row[11:]:
                if s == '#':
                    e = simulation_vars[row[1]].eff
                    pol = False
                elif s == "" or s == " ":
                    continue
                else:
                    a = separateByC(s)
                    e_vars = separateEffect(a[1])
                    if len(e_vars) == 5:
                        if not pol:
                            e.append(Effect(row[1], a[0], e_vars[0], e_vars[1], e_vars[2], e_vars[3], e_vars[4]))
                        else:
                            e.append(Effect(a[0], a[0], e_vars[0], e_vars[1], e_vars[2], e_vars[3], e_vars[4]))
                    elif len(e_vars) == 7:
                        if not pol:
                            e.append(Effect(row[1], a[0], e_vars[0], e_vars[1], e_vars[2], e_vars[3], e_vars[4], e_vars[5], e_vars[6]))
                        else:
                            e.append(Effect(a[0], a[0], e_vars[0], e_vars[1], e_vars[2], e_vars[3], e_vars[4]))
        else:
            i += 1

In [164]:
voter_types = {}

# Implement influences

with open("votertypes.csv") as rs:
    csv_reader = csv.reader(rs, delimiter=',')
    i = 0
    for row in csv_reader:
#         for i in range(len(row)):
#             print(i ,row[i])
        if i > 0:
            voter_types[row[1]] = float(row[6])
            voter_types[row[1]+"_perc"] = float(row[7])
        else:
            i += 1

In [165]:
situations = {}

#positive	start trigger	stop trigger	Cost	Income
class Situations(All):
    def __init__(self, positive, start, stop, cost, income):
        self.positive = float(positive)
        self.start = float(start)
        self.stop = float(stop)
        self.cost = float(cost)
        self.started = False
        self.prereq = []
        self.income = float(income)
        self.default = 0.0
        self.aff = []
        self.eff = []
        self.value = {0.0: 0.0}
        self.effect = {0.0: 1.0}
    def realCost(self):
        return self.cost*all_vars['wealth_mod'] if self.started else 0.0
    def realIncome(self):
        return self.income*all_vars['wealth_mod'] if self.started else 0.0
    def getValue(self, count=1):
        if not self.started:
            return 0.0
        avg = 0
        i = all_vars['_year']
        while i > all_vars['_year'] - count/4:
            if i in self.value:
                avg += self.value[i]*self.effect[i]
            i -= 1/4
        return avg/count
    def applyEffects(self):
        if self.started:
            for i in self.eff:
                # apply effects
                setEffect(i.target, i.apply(), True)
    def selfStarter(self):
        yr = max(0.0, all_vars['_year']- 1/4)
        if self.value[yr]*self.effect[yr] > self.stop:
            if self.value[yr]*self.effect[yr] > self.start:
                self.started = True
        else:
            self.started = False
    def addValue(self, value, nxt=False):
        if nxt:
            yr = all_vars['_year'] + 1/4
        else:
            yr = all_vars['_year']
        if yr in self.value:
            self.value[yr] += float(value)
        else:
            self.value[yr] = self.default + float(value)
    def update(self):
        # Code for Updating Aff and Eff
        # Check prereqs ****************
        #self.value[all_vars['_year']] = self.value[all_vars['_year']-1/4]
        self.effect[all_vars['_year']] = 1.0
        self.value[all_vars['_year']] = self.default
        for i in self.aff:
            self.addValue(i.apply())
        self.selfStarter()
        if self.started:
            self.applyEffects()
            # Need to use effects code here

with open("situations.csv") as rs:
    csv_reader = csv.reader(rs, delimiter=',')
    i = 0
    for row in csv_reader:
        if i > 0:
            situations[row[1]] = Situations(row[8], row[9], row[10], row[11], row[12])
            e = situations[row[1]].aff
            pol = True
            for s in row[13:]:
                a = separateByC(s)
                if a[0] == "_default_":
                    situations[row[1]].default = float(a[1])
                    situations[row[1]].setValue(float(a[1]))
                elif a[0] == "_prereq_":
                    situations[row[1]].prereq.append(a[1])
                elif s == '#':
                    e = situations[row[1]].eff
                elif s == "" or s == " ":
                    continue
                else:
                    if len(e_vars) == 5:
                        if not pol:
                            e.append(Effect(row[1], a[0], e_vars[0], e_vars[1], e_vars[2], e_vars[3], e_vars[4]))
                        else:
                            e.append(Effect(a[0], a[0], e_vars[0], e_vars[1], e_vars[2], e_vars[3], e_vars[4]))
                    elif len(e_vars) == 7:
                        if not pol:
                            e.append(Effect(row[1], a[0], e_vars[0], e_vars[1], e_vars[2], e_vars[3], e_vars[4], e_vars[5], e_vars[6]))
                        else:
                            e.append(Effect(a[0], a[0], e_vars[0], e_vars[1], e_vars[2], e_vars[3], e_vars[4]))
        else:
            i += 1

In [166]:
# voter_types, simulation_vars, policy_names, config, slider_names
for i in voter_types:
    all_vars[i] = voter_types[i]
for i in simulation_vars:
    all_vars[i] = simulation_vars[i]
for i in policy_names:
    all_vars[i] = policy_names[i]
for i in config:
    all_vars[i] = config[i]
for i in slider_names:
    all_vars[i] = slider_names[i]
for i in situations:
    all_vars[i] = situations[i]

# print(all_vars)

# Create a new grudge object

# import glob

# for filename in glob.glob('grudges/*.txt'):
#     with open(filename) as rs:
#         for ln in rs:
#             # INSERT SETTER CODE HERE
#             commaCount = 0
#             for letter in ln:
#                 if commaCount == 2:
                    
#                 elif commaCount == 3:
                    
#                 if letter == ',':
#                     commaCount+= 1

In [167]:
with open("usa.txt") as rs:
    j = 0
    for ln in rs:
        if ln == "[policies]\n":
            j = 1
        elif j == 1:
            i = 0
            var = ""
            while i < len(ln) and ln[i] != " ":
                var += ln[i]
                i += 1
            i += 3
            num = ""
            while i < len(ln):
                num += ln[i]
                i += 1
            policy_names[var].value[all_vars['_year']] = float(num)
            policy_names[var].effect[all_vars['_year']] = 1.0
            policy_names[var].activate(1)
with open("usa.txt") as rs:
    j = 0
    for ln in rs:
        j+=1
        if j >= 4 and j <= 9:
            i = 0
            var = ""
            while i < len(ln) and ln[i] != " ":
                var += ln[i]
                i += 1
            i += 3
            num = ""
            while i < len(ln):
                num += ln[i]
                i += 1
            all_vars[var] = float(num)
    all_vars['_effectivedebt_'] = 3120000.0
    all_vars['GDP'].realMaxim = 6063280.0
    all_vars['GDP'].realMinim = 2021093.0

In [168]:
def debtToGDP():
    return min(all_vars['DEBT_TO_GDP_MAX'], all_vars['_effectivedebt_']/all_vars['GDP'].getRealValue())

# CREDIT_RATING_A = 0.4
# CREDIT_RATING_AA = 0.35
# CREDIT_RATING_AAA = 0.25
# CREDIT_RATING_B = 0.6
# CREDIT_RATING_BB = 0.5
# CREDIT_RATING_BBB = 0.45
# CREDIT_RATING_C = 0.9
# CREDIT_RATING_CC = 0.8
# CREDIT_RATING_CCC = 0.7

#  12.21 at C
#  9.75 at CC
#  7.61 at CCC
#  5.8 at B
#  4.33 at BB
#  3.18 at BBB
#  2.36 at A
#  1.86 at AA
#  1.7 at AAA

class InterestRate():
    interest = 0.0
    def __init__():
        InterestRate.interest = findInterest()
    def applyInterest():
        return all_vars['_effectivedebt_']*InterestRate.interest/4
    def update():
        InterestRate.interest = InterestRate.findInterest()
    def getValue(inertia=1):
        # FIX INERTIA
        return InterestRate.interest
    def findInterest():
        if debtToGDP() <= 0.5:
            #AAA
            return 0.017
        elif debtToGDP() <= 0.7:
            #AA
            return 0.0186
        elif debtToGDP() <= 0.8:
            #A
            return 0.0236
        elif debtToGDP() <= 0.9:
            #BBB
            return 0.0318
        elif debtToGDP() <= 1.0:
            #BB
            return 0.0433
        elif debtToGDP() <= 1.2:
            #B
            return 0.058
        elif debtToGDP() <= 1.4:
            #CCC
            return 0.0761
        elif debtToGDP() <= 1.6:
            #CC
            return 0.0975
        else:
            # if debtToGDP() <= 1.8:
            #C
            return 0.1221
all_vars['_global_interest_rates_'] = InterestRate

In [169]:
from math import *

# You need to add the adder function to these classes

class EconomicCycle():
    def __init__(self, current_time):
        self.current_time = current_time
    def update(self):
        self.current_time += 1
    def setValue(self, value):
        self.current_time = value
    def addValue(self, value):
        self.value += value
    def getValue(self, inertia=1):
        # Implement Inertia
        return sin(self.current_time*2*pi/all_vars['GLOBAL_ECONOMY_CYCLE_LENGTH_YEARS'])*all_vars["GLOBAL_ECONOMY_INTENSITY"]

all_vars["_globaleconomy_"] = EconomicCycle(all_vars["economic_cycle_start"]*all_vars['GLOBAL_ECONOMY_CYCLE_LENGTH_YEARS'])
# all_vars["_globaleconomy_"].getValue()

In [170]:
def updater():
    # Run all the functions
    income = 0.0
    cost = 0.0
#     print('Policies')
    for i in active_policies:
        all_vars[i].incomeMul()
        all_vars[i].costMul()
        all_vars[i].applyEffects()
#     print('Situations')
    for i in situations:
        income += all_vars[i].realIncome()
        cost += all_vars[i].realCost()
        all_vars[i].applyEffects()
    for i in active_policies:
        income += all_vars[i].realIncome()
        cost += all_vars[i].realCost()
    cost += all_vars['_global_interest_rates_'].applyInterest()
    all_vars['_effectivedebt_']  += (cost - income)
#     for i in situations:
#         print(i, all_vars[i].realIncome(), all_vars[i].realCost(), all_vars[i].getValue())
    for i in active_policies:
        print(i, all_vars[i].realIncome(), all_vars[i].realCost(),)
#         print(i, all_vars[i].realIncome(), all_vars[i].income_multiply, all_vars[i].cost_multiply, all_vars[i].value)
#         print(i, all_vars[i].realIncome(), all_vars[i].realCost(), all_vars[i].getValue())
#     for i in simulation_vars:
#         print(i, all_vars[i].getValue())
    all_vars['_year'] += 0.25
    all_vars["_globaleconomy_"].update()
    for i in situations:
        situations[i].update()
    for i in simulation_vars:
        simulation_vars[i].update()
    for i in policy_names:
        all_vars[i].update()
    print(all_vars['GDP'].getValue(), all_vars['GDP'].effect, all_vars['GDP'].value)
    print('ECON', all_vars["_globaleconomy_"].getValue())
    if int(all_vars['_year']*4)%2 == 0:
        all_vars['_global_interest_rates_'].update()
    print("Interesting", InterestRate.interest)
    print(all_vars['_year'], income, cost, income - cost, all_vars['_effectivedebt_'])

In [171]:
all_vars['Young_perc'] += 0.005
all_vars['Socialist_perc'] += -0.12
all_vars['Farmers_perc'] += 0.035
all_vars['Religious_perc'] += 0.33
all_vars['Retired_perc'] += 0.08
all_vars['DrugAddiction'].addValue(0.26)
all_vars['ClassWarfare'].addValue(-0.18)
all_vars['GeneralStrike'].addValue(-0.18)
all_vars['Obesity'].addValue(0.13)
for i in situations:
    situations[i].update()

all_vars['_global_interest_rates_'].update()

for _ in range(200):
    updater()

AlcoholLaw 0.0 0.0
BorderControls 0.0 2762.38
Creationism 0.0 0.0
HandgunLaws 0.0 18.728
IntelligenceServices 0.0 33850.86
LabourLaws 0.0 2481.46
MilitarySpending 0.0 234802.3
Narcotics 0.0 0.0
PoliceForce 0.0 33291.595100000006
Prisons 0.0 35360.805
RoadBuilding 0.0 60866.0
ScienceFunding 0.0 47814.925
AbortionLaw 0.0 0.0
CorporationTax 150698.211335 0.0
SalesTax 212293.2141856 0.0
PetrolTax 46296.36512 0.0
UnemployedBenefit 0.0 27247.741760000004
StateHealthService 0.0 85526.094
StateSchools 0.0 96528.794
PropertyTax 72970.72575 0.0
RailSubsidies 0.0 57003.35
TobaccoTax 36221.532175 0.0
ArmedPolice 0.0 15984.347999999998
StatePensions 0.0 196307.84283127534
PollutionControls 0.0 342.2542
FoodStamps 0.0 93640.0
PrivatePrisons 0.0 6759.459583999999
TechnologyColleges 0.0 13641.007000000001
TechnologyGrants 0.0 107686.0
SpaceProgram 0.0 74326.75
SchoolBuses 0.0 10593.025
LegalAid 0.0 4799.05
JuryTrial 0.0 6166.428099999999
HybridCarsInitiative 0.0 1543.596875
GatedCommunities 0.0 0.0
Fo

PropertyTax 72970.72575 0.0
RailSubsidies 0.0 57003.35
TobaccoTax 36221.532175 0.0
ArmedPolice 0.0 15984.347999999998
StatePensions 0.0 196307.84283127534
PollutionControls 0.0 342.2542
FoodStamps 0.0 93640.0
PrivatePrisons 0.0 6759.459583999999
TechnologyColleges 0.0 13641.007000000001
TechnologyGrants 0.0 107686.0
SpaceProgram 0.0 74326.75
SchoolBuses 0.0 10593.025
LegalAid 0.0 4799.05
JuryTrial 0.0 6166.428099999999
HybridCarsInitiative 0.0 1543.596875
GatedCommunities 0.0 0.0
ForeignAid 0.0 16475.958
DeathPenalty 0.0 0.0
BiofuelSubsidies 0.0 16387.0
IncomeTax 656048.403564704 0.0
CapitalGainsTax 50734.607057626 0.0
CarTax 9273.2291296 0.0
InheritanceTax 16497.4952 0.0
PrisonerTagging 0.0 580.568
SmallBusinessGrants 0.0 117050.0
CitizenshipTests 0.0 485.42975999999993
SchoolVouchers 0.0 50640.511999999995
FoodStandards 0.0 1038.4676
0.3781071971878586 {0.0: 1.0, 0.25: 1.0, 0.5: 1.0, 0.75: 1.0, 1.0: 1.0, 1.25: 1.0, 1.5: 1.0, 1.75: 1.9591120999227563} {0.0: 0.35, 0.25: 0.4343125066778

BorderControls 0.0 2762.38
Creationism 0.0 0.0
HandgunLaws 0.0 18.728
IntelligenceServices 0.0 33850.86
LabourLaws 0.0 2481.46
MilitarySpending 0.0 234802.3
Narcotics 0.0 0.0
PoliceForce 0.0 33291.595100000006
Prisons 0.0 35360.805
RoadBuilding 0.0 60866.0
ScienceFunding 0.0 47814.925
AbortionLaw 0.0 0.0
CorporationTax 142463.96365489997 0.0
SalesTax 199965.60140609278 0.0
PetrolTax 46296.36512 0.0
UnemployedBenefit 0.0 27247.741760000004
StateHealthService 0.0 95725.64499944591
StateSchools 0.0 96528.794
PropertyTax 72970.72575 0.0
RailSubsidies 0.0 57003.35
TobaccoTax 36221.532175 0.0
ArmedPolice 0.0 15984.347999999998
StatePensions 0.0 196307.84283127534
PollutionControls 0.0 342.2542
FoodStamps 0.0 93640.0
PrivatePrisons 0.0 6759.459583999999
TechnologyColleges 0.0 13641.007000000001
TechnologyGrants 0.0 107686.0
SpaceProgram 0.0 74326.75
SchoolBuses 0.0 10593.025
LegalAid 0.0 4799.05
JuryTrial 0.0 6166.428099999999
HybridCarsInitiative 0.0 1543.596875
GatedCommunities 0.0 0.0
Fore

PoliceForce 0.0 33291.595100000006
Prisons 0.0 35360.805
RoadBuilding 0.0 60866.0
ScienceFunding 0.0 47814.925
AbortionLaw 0.0 0.0
CorporationTax 142463.96365489997 0.0
SalesTax 199965.60140609278 0.0
PetrolTax 46296.36512 0.0
UnemployedBenefit 0.0 27247.741760000004
StateHealthService 0.0 95800.1499157541
StateSchools 0.0 96528.794
PropertyTax 72970.72575 0.0
RailSubsidies 0.0 57003.35
TobaccoTax 36221.532175 0.0
ArmedPolice 0.0 15984.347999999998
StatePensions 0.0 196307.84283127534
PollutionControls 0.0 342.2542
FoodStamps 0.0 93640.0
PrivatePrisons 0.0 6759.459583999999
TechnologyColleges 0.0 13641.007000000001
TechnologyGrants 0.0 107686.0
SpaceProgram 0.0 74326.75
SchoolBuses 0.0 10593.025
LegalAid 0.0 4799.05
JuryTrial 0.0 6166.428099999999
HybridCarsInitiative 0.0 1543.596875
GatedCommunities 0.0 0.0
ForeignAid 0.0 16475.958
DeathPenalty 0.0 0.0
BiofuelSubsidies 0.0 16387.0
IncomeTax 656048.403564704 0.0
CapitalGainsTax 50734.607057626 0.0
CarTax 9273.2291296 0.0
InheritanceTax

UnemployedBenefit 0.0 27247.741760000004
StateHealthService 0.0 95725.64499944591
StateSchools 0.0 96528.794
PropertyTax 72970.72575 0.0
RailSubsidies 0.0 57003.35
TobaccoTax 36221.532175 0.0
ArmedPolice 0.0 15984.347999999998
StatePensions 0.0 196307.84283127534
PollutionControls 0.0 342.2542
FoodStamps 0.0 93640.0
PrivatePrisons 0.0 6759.459583999999
TechnologyColleges 0.0 13641.007000000001
TechnologyGrants 0.0 107686.0
SpaceProgram 0.0 74326.75
SchoolBuses 0.0 10593.025
LegalAid 0.0 4799.05
JuryTrial 0.0 6166.428099999999
HybridCarsInitiative 0.0 1543.596875
GatedCommunities 0.0 0.0
ForeignAid 0.0 16475.958
DeathPenalty 0.0 0.0
BiofuelSubsidies 0.0 16387.0
IncomeTax 656048.403564704 0.0
CapitalGainsTax 50734.607057626 0.0
CarTax 9273.2291296 0.0
InheritanceTax 16497.4952 0.0
PrisonerTagging 0.0 580.568
SmallBusinessGrants 0.0 117050.0
CitizenshipTests 0.0 485.42975999999993
SchoolVouchers 0.0 50640.511999999995
FoodStandards 0.0 1038.4676
0.3781071971878589 {0.0: 1.0, 0.25: 1.0, 0.

...

SchoolBuses 0.0 10593.025
LegalAid 0.0 4799.05
JuryTrial 0.0 6166.428099999999
HybridCarsInitiative 0.0 1543.596875
GatedCommunities 0.0 0.0
ForeignAid 0.0 16475.958
DeathPenalty 0.0 0.0
BiofuelSubsidies 0.0 16387.0
IncomeTax 656048.403564704 0.0
CapitalGainsTax 50734.607057626 0.0
CarTax 9273.2291296 0.0
InheritanceTax 16497.4952 0.0
PrisonerTagging 0.0 580.568
SmallBusinessGrants 0.0 117050.0
CitizenshipTests 0.0 485.42975999999993
SchoolVouchers 0.0 50640.511999999995
FoodStandards 0.0 1038.4676
0.20265691239069678 {0.0: 1.0, 0.25: 1.0, 0.5: 1.0, 0.75: 1.0, 1.0: 1.0, 1.25: 1.0, 1.5: 1.0, 1.75: 1.0, 2.0: 1.0, 2.25: 1.0, 2.5: 1.0, 2.75: 1.0, 3.0: 1.0, 3.25: 1.0, 3.5: 1.0, 3.75: 1.0, 4.0: 1.0, 4.25: 1.0, 4.5: 1.0, 4.75: 1.0, 5.0: 1.0, 5.25: 1.0, 5.5: 1.0, 5.75: 1.0, 6.0: 1.0, 6.25: 1.0, 6.5: 1.0, 6.75: 1.0, 7.0: 1.0, 7.25: 1.0, 7.5: 1.0, 7.75: 1.0, 8.0: 1.0, 8.25: 1.0, 8.5: 1.0, 8.75: 1.0, 9.0: 1.0, 9.25: 1.0, 9.5: 1.0, 9.75: 1.0, 10.0: 1.0, 10.25: 1.0, 10.5: 1.0, 10.75: 1.0, 11.0: 1.0

CitizenshipTests 0.0 485.42975999999993
SchoolVouchers 0.0 50640.511999999995
FoodStandards 0.0 1038.4676
0.22593791385881584 {0.0: 1.0, 0.25: 1.0, 0.5: 1.0, 0.75: 1.0, 1.0: 1.0, 1.25: 1.0, 1.5: 1.0, 1.75: 1.0, 2.0: 1.0, 2.25: 1.0, 2.5: 1.0, 2.75: 1.0, 3.0: 1.0, 3.25: 1.0, 3.5: 1.0, 3.75: 1.0, 4.0: 1.0, 4.25: 1.0, 4.5: 1.0, 4.75: 1.0, 5.0: 1.0, 5.25: 1.0, 5.5: 1.0, 5.75: 1.0, 6.0: 1.0, 6.25: 1.0, 6.5: 1.0, 6.75: 1.0, 7.0: 1.0, 7.25: 1.0, 7.5: 1.0, 7.75: 1.0, 8.0: 1.0, 8.25: 1.0, 8.5: 1.0, 8.75: 1.0, 9.0: 1.0, 9.25: 1.0, 9.5: 1.0, 9.75: 1.0, 10.0: 1.0, 10.25: 1.0, 10.5: 1.0, 10.75: 1.0, 11.0: 1.0, 11.25: 1.0, 11.5: 1.0, 11.75: 1.0, 12.0: 1.0, 12.25: 1.0, 12.5: 1.0, 12.75: 1.0, 13.0: 1.0, 13.25: 1.0, 13.5: 1.0, 13.75: 1.0, 14.0: 1.0, 14.25: 1.0, 14.5: 1.0, 14.75: 1.0, 15.0: 1.0, 15.25: 1.0, 15.5: 1.0, 15.75: 1.0, 16.0: 1.0, 16.25: 1.0, 16.5: 1.0, 16.75: 1.0, 17.0: 1.0, 17.25: 1.0, 17.5: 1.0, 17.75: 1.0, 18.0: 1.0, 18.25: 1.0, 18.5: 1.0, 18.75: 1.0, 19.0: 1.0, 19.25: 1.0, 19.5: 1.0, 19.75

AlcoholLaw 0.0 0.0
BorderControls 0.0 2762.38
Creationism 0.0 0.0
HandgunLaws 0.0 18.728
IntelligenceServices 0.0 33850.86
LabourLaws 0.0 2481.46
MilitarySpending 0.0 234802.3
Narcotics 0.0 0.0
PoliceForce 0.0 33291.595100000006
Prisons 0.0 35360.805
RoadBuilding 0.0 60866.0
ScienceFunding 0.0 47814.925
AbortionLaw 0.0 0.0
CorporationTax 142463.96365489997 0.0
SalesTax 199965.60140609278 0.0
PetrolTax 46296.36512 0.0
UnemployedBenefit 0.0 27247.741760000004
StateHealthService 0.0 95755.7911711494
StateSchools 0.0 96528.794
PropertyTax 72970.72575 0.0
RailSubsidies 0.0 57003.35
TobaccoTax 36221.532175 0.0
ArmedPolice 0.0 15984.347999999998
StatePensions 0.0 196307.84283127534
PollutionControls 0.0 342.2542
FoodStamps 0.0 93640.0
PrivatePrisons 0.0 6759.459583999999
TechnologyColleges 0.0 13641.007000000001
TechnologyGrants 0.0 107686.0
SpaceProgram 0.0 74326.75
SchoolBuses 0.0 10593.025
LegalAid 0.0 4799.05
JuryTrial 0.0 6166.428099999999
HybridCarsInitiative 0.0 1543.596875
GatedCommun

TobaccoTax 36221.532175 0.0
ArmedPolice 0.0 15984.347999999998
StatePensions 0.0 196307.84283127534
PollutionControls 0.0 342.2542
FoodStamps 0.0 93640.0
PrivatePrisons 0.0 6759.459583999999
TechnologyColleges 0.0 13641.007000000001
TechnologyGrants 0.0 107686.0
SpaceProgram 0.0 74326.75
SchoolBuses 0.0 10593.025
LegalAid 0.0 4799.05
JuryTrial 0.0 6166.428099999999
HybridCarsInitiative 0.0 1543.596875
GatedCommunities 0.0 0.0
ForeignAid 0.0 16475.958
DeathPenalty 0.0 0.0
BiofuelSubsidies 0.0 16387.0
IncomeTax 656048.403564704 0.0
CapitalGainsTax 50734.607057626 0.0
CarTax 9273.2291296 0.0
InheritanceTax 16497.4952 0.0
PrisonerTagging 0.0 580.568
SmallBusinessGrants 0.0 117050.0
CitizenshipTests 0.0 485.42975999999993
SchoolVouchers 0.0 50640.511999999995
FoodStandards 0.0 1038.4676
0.20265691239069655 {0.0: 1.0, 0.25: 1.0, 0.5: 1.0, 0.75: 1.0, 1.0: 1.0, 1.25: 1.0, 1.5: 1.0, 1.75: 1.0, 2.0: 1.0, 2.25: 1.0, 2.5: 1.0, 2.75: 1.0, 3.0: 1.0, 3.25: 1.0, 3.5: 1.0, 3.75: 1.0, 4.0: 1.0, 4.25: 1.

Creationism 0.0 0.0
HandgunLaws 0.0 18.728
IntelligenceServices 0.0 33850.86
LabourLaws 0.0 2481.46
MilitarySpending 0.0 234802.3
Narcotics 0.0 0.0
PoliceForce 0.0 33291.595100000006
Prisons 0.0 35360.805
RoadBuilding 0.0 60866.0
ScienceFunding 0.0 47814.925
AbortionLaw 0.0 0.0
CorporationTax 142463.96365489997 0.0
SalesTax 199965.60140609278 0.0
PetrolTax 46296.36512 0.0
UnemployedBenefit 0.0 27247.741760000004
StateHealthService 0.0 95755.7911711494
StateSchools 0.0 96528.794
PropertyTax 72970.72575 0.0
RailSubsidies 0.0 57003.35
TobaccoTax 36221.532175 0.0
ArmedPolice 0.0 15984.347999999998
StatePensions 0.0 196307.84283127534
PollutionControls 0.0 342.2542
FoodStamps 0.0 93640.0
PrivatePrisons 0.0 6759.459583999999
TechnologyColleges 0.0 13641.007000000001
TechnologyGrants 0.0 107686.0
SpaceProgram 0.0 74326.75
SchoolBuses 0.0 10593.025
LegalAid 0.0 4799.05
JuryTrial 0.0 6166.428099999999
HybridCarsInitiative 0.0 1543.596875
GatedCommunities 0.0 0.0
ForeignAid 0.0 16475.958
DeathPe

In [172]:
all_vars['_year'] = 0.25
print(all_vars['Pollution'].value)
print(all_vars['IncomeTax'].value[0.0],all_vars['IncomeTax'].realIncome())
print(all_vars["GeneralStrike"].getValue(1))

# Fix Situations  *
# Fix Inertia *
# Economic Cycles *
# Add Events *
# Implement Grudges
# Implement dilemmas and events *
# People Groups
# Overall Happiness/Voting

{0.0: 0.0, 0.25: 0.12, 0.5: 0.12, 0.75: 0.12, 1.0: 0.12, 1.25: 0.12, 1.5: 0.12, 1.75: 0.12, 2.0: 0.12, 2.25: 0.12, 2.5: 0.12, 2.75: 0.12, 3.0: 0.12, 3.25: 0.12, 3.5: 0.12, 3.75: 0.12, 4.0: 0.12, 4.25: 0.12, 4.5: 0.12, 4.75: 0.12, 5.0: 0.12, 5.25: 0.12, 5.5: 0.12, 5.75: 0.12, 6.0: 0.12, 6.25: 0.12, 6.5: 0.12, 6.75: 0.12, 7.0: 0.12, 7.25: 0.12, 7.5: 0.12, 7.75: 0.12, 8.0: 0.12, 8.25: 0.12, 8.5: 0.12, 8.75: 0.12, 9.0: 0.12, 9.25: 0.12, 9.5: 0.12, 9.75: 0.12, 10.0: 0.12, 10.25: 0.12, 10.5: 0.12, 10.75: 0.12, 11.0: 0.12, 11.25: 0.12, 11.5: 0.12, 11.75: 0.12, 12.0: 0.12, 12.25: 0.12, 12.5: 0.12, 12.75: 0.12, 13.0: 0.12, 13.25: 0.12, 13.5: 0.12, 13.75: 0.12, 14.0: 0.12, 14.25: 0.12, 14.5: 0.12, 14.75: 0.12, 15.0: 0.12, 15.25: 0.12, 15.5: 0.12, 15.75: 0.12, 16.0: 0.12, 16.25: 0.12, 16.5: 0.12, 16.75: 0.12, 17.0: 0.12, 17.25: 0.12, 17.5: 0.12, 17.75: 0.12, 18.0: 0.12, 18.25: 0.12, 18.5: 0.12, 18.75: 0.12, 19.0: 0.12, 19.25: 0.12, 19.5: 0.12, 19.75: 0.12, 20.0: 0.12, 20.25: 0.12, 20.5: 0.12, 20.